In [2]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

import random
import pickle
import gc
from tqdm import tqdm

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf

from keras import layers, losses, optimizers, Model

2023-11-12 21:51:16.234101: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-12 21:51:16.234177: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-12 21:51:16.234229: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
gc.collect()

0

In [12]:
def get_feature_vectors():
    with open("coeff_vecs.txt", "r") as f:
        f = f.read().split("\n\n\n")
        noun_dict = {}
        for noun_vec in f:
            split = noun_vec.split("\n\n")
            noun = split[0].split()[2][:-1]
            
            vec = split[1]
            vec = [item.strip().replace("(", "").replace(")", "") for item in vec.split(",\n")]
            vec = [(item.split()[:-1], item.split()[-1]) for item in vec]

            vec = sorted(vec, key=lambda x: x[0])
            vec = {" ".join(item[0]): float(item[1]) for item in vec}

            noun_dict[noun] = vec

    return {k: [noun_dict[k][k1] for k1 in noun_dict[k]] for k in noun_dict}

In [18]:
feature_vectors = get_feature_vectors()
nouns, verbs = pickle.load(open("vecs.pkl", "rb"))

XY = [(feature_vectors[noun], nouns[noun]) for noun in nouns]

In [6]:
class BasisSum(Model):
    def __init__(self):
        super().__init__()
        self.basis = tf.Variable(tf.zeros((25, 50)))

    @tf.function(reduce_retracing=True)
    def call(self, x):
        x = tf.einsum("ij,bi->bj", self.basis, x)

        return x

In [7]:
def l2(a, b):
    return -norm(np.subtract(a, b))

In [8]:
total = 500
pbar = tqdm(range(total))
correct_count = 0

for i in pbar:
    model = BasisSum()

    loss = losses.MeanSquaredError()
    opt = optimizers.Adam(0.01)
    
    random.shuffle(XY)

    X, Y = tf.cast(X, tf.dtypes.float32), tf.cast(Y, tf.dtypes.float32)
    
    train_x, test_x = X[:-2], X[-2:]
    train_y, test_y = Y[:-2], Y[-2:]

    for j in range(100):
        with tf.GradientTape() as tape:
            pred_y = model(train_x)
            batchloss = loss(train_y, pred_y)

            grad = tape.gradient(batchloss, model.trainable_variables)
            opt.apply_gradients(zip(grad, model.trainable_variables))


    pred = model(test_x)
    t1, t2 = test_y.numpy()
    t1, t2 = t1.flat, t2.flat
    p1, p2 = pred.numpy()
    p1, p2 = p1.flat, p2.flat
    
    correct = l2(t1, p1) + l2(t2, p2) > l2(t1, p2) + l2(t2, p1)
    correct_count += int(correct)

    pbar.set_description(f"accuracy: {correct_count / (i + 1):.3f}")

  0%|          | 0/500 [00:00<?, ?it/s]


KeyError: 51